In [1]:
import os
from pathlib import Path

import emv_tools
import emv_tools.ffi.scipion as scipion
from emv_tools.ffi.blocres import blocres
from emv_tools.scipion_bridge.proxy import OutputInfo

from utils import download as D
emv_tools.scipion_bridge.environment.configure_default_env()

## Download Data

In [2]:
DOWNLOAD_PATH = Path("data/downloaded")
EMDB_ENTRY = 41510

In [3]:
metadata = D.download_emdb_metadata(entry_id=EMDB_ENTRY)

os.makedirs(DOWNLOAD_PATH, exist_ok=True)

emdb_map = D.download_emdb_map(EMDB_ENTRY, DOWNLOAD_PATH)
map_1, map_2 = D.download_halfmaps(EMDB_ENTRY, DOWNLOAD_PATH)
pdb_model = D.download_pdb_model(metadata.pdb_id, DOWNLOAD_PATH)

metadata

EMDBMetadata(pdb_id='7l70', resolution=3.1, sampling=0.835, size=256, org_x=0, org_y=0, org_z=0)

## Create Volume from PDB

In [4]:
volume = scipion.xmipp_volume_from_pdb(
    pdb_model,
    OutputInfo(None),
    center_pdb="-v 0",
    sampling=metadata.sampling,
    size=metadata.size,
).reassign("vol")

volume = scipion.xmipp_volume_align(
    embdb_map=str(emdb_map),
    volume=volume,
    local=True
)

[XMIPP] scipion run xmipp_volume_from_pdb -i data/downloaded/pdb7l70.ent -o /tmp/tmp24aq2z7f --centerPDB -v 0 --sampling 0.835 --size 256
Xmipp command detected
Scipion v3.7.1 - Eugenius
[XMIPP] scipion run xmipp_volume_align --i1 data/downloaded/emd_41510.map --i2 /tmp/tmp24aq2z7f.vol --local --apply /tmp/tmp0njy6lcj.vol
1 (1,0,0,0,0,1,0,0,0)--->-0.183028
   (1,0,***3.01869,0,0,1,0,0,0)--->-0.188881
   (1,0,3.01869,***0.58786,0,1,0,0,0)--->-0.189913
   (1,0,3.01869,0.58786,***0.144014,1,0,0,0)--->-0.189927
   (1,0,3.01869,0.58786,0.144014,***0.837076,0,0,0)--->-0.204567
   (1,0,3.01869,0.58786,0.144014,0.837076,***0.604711,0,0)--->-0.204677
   (1,0,3.01869,0.58786,0.144014,0.837076,0.604711,***10.2284,0)--->-0.214948
   (1,0,3.01869,0.58786,0.144014,0.837076,0.604711,10.2284,***-0.56966)--->-0.214975
2 (1,0,3.01869,0.58786,0.144014,0.837076,0.604711,10.2284,-0.56966)--->-0.214975
   (1,0,***5.63938,0.58786,0.144014,0.837076,0.604711,10.2284,-0.56966)--->-0.21675
   (1,0,5.63938,***-15

### Mask from Volume

In [5]:
mask = scipion.xmipp_transform_threshold(
    volume, select="below 0.02", substitute="binarize"
)
mask = scipion.xmipp_transform_morphology(mask, binary_operation="dilation", size=2)

mask

[XMIPP] scipion run xmipp_transform_threshold -i /tmp/tmp0njy6lcj.vol -o /tmp/tmp2qv4hzie.vol --select below 0.02 --substitute binarize
Input File: /tmp/tmp0njy6lcj.vol
Output File: /tmp/tmp2qv4hzie.vol
Xmipp command detected
Scipion v3.7.1 - Eugenius
[XMIPP] scipion run xmipp_transform_morphology -i /tmp/tmp2qv4hzie.vol -o /tmp/tmpf2mz8p0r.vol --binaryOperation dilation --size 2
Input File: /tmp/tmp2qv4hzie.vol
Output File: /tmp/tmpf2mz8p0r.vol
Initially the image has 623800 pixels set to 1
Finally the image has 843172 pixels set to 1
Xmipp command detected
Scipion v3.7.1 - Eugenius


Output()

In [6]:
def apply_sampling(data, sampling):
    data = scipion.xmipp_image_convert(data)
    scipion.xmipp_image_header(data, sampling=sampling)

    return data

volume = apply_sampling(volume, metadata.sampling)
mask = apply_sampling(mask, metadata.sampling)

[XMIPP] scipion run xmipp_image_convert -i /tmp/tmp0njy6lcj.vol -o /tmp/tmplq9uqje8.mrc -t vol
Input File: /tmp/tmp0njy6lcj.vol
Output File: /tmp/tmplq9uqje8.mrc
Xmipp command detected
Scipion v3.7.1 - Eugenius
[XMIPP] scipion run xmipp_image_header -i /tmp/tmplq9uqje8.mrc -s 0.835
Input File: /tmp/tmplq9uqje8.mrc
New sampling rate (Angstrom) = 0.835
Xmipp command detected
Scipion v3.7.1 - Eugenius
[XMIPP] scipion run xmipp_image_convert -i /tmp/tmpf2mz8p0r.vol -o /tmp/tmpksqi23ou.mrc -t vol
Input File: /tmp/tmpf2mz8p0r.vol
Output File: /tmp/tmpksqi23ou.mrc
Xmipp command detected
Scipion v3.7.1 - Eugenius
[XMIPP] scipion run xmipp_image_header -i /tmp/tmpksqi23ou.mrc -s 0.835
Input File: /tmp/tmpksqi23ou.mrc
New sampling rate (Angstrom) = 0.835
Xmipp command detected
Scipion v3.7.1 - Eugenius


## Compute BlocRes

In [7]:
blocres_half = blocres(
    str(map_1),
    str(map_2),
    mask=mask,
    sampling=f"{metadata.sampling},{metadata.sampling},{metadata.sampling}",
    box=int(metadata.resolution * 5),
    cutoff=0.5,
    step=5,
)

[BlocRes] /home/max/scipion3/software/em/bsoft/bin/blocres  -Mask /tmp/tmpksqi23ou.mrc -sampling 0.835,0.835,0.835 -box 15 -cutoff 0.5 -nofill -smooth -pad 1 -step 5 -maxresolution 0.5 -verbose 0 data/downloaded/emd_41510_half_map_1.map data/downloaded/emd_41510_half_map_2.map /tmp/tmp7t3wchsc.map


In [8]:
blocres_model = blocres(
    str(emdb_map),
    volume,
    mask=mask,
    sampling=f"{metadata.sampling},{metadata.sampling},{metadata.sampling}",
    box=int(metadata.resolution * 5),
    cutoff=0.67,
    step=5,
)

[BlocRes] /home/max/scipion3/software/em/bsoft/bin/blocres  -Mask /tmp/tmpksqi23ou.mrc -sampling 0.835,0.835,0.835 -box 15 -cutoff 0.67 -nofill -smooth -pad 1 -step 5 -maxresolution 0.5 -verbose 0 data/downloaded/emd_41510.map /tmp/tmplq9uqje8.mrc /tmp/tmp_yga7l_h.map


## Compute FSCQ

In [11]:
fscq_diff = scipion.xmipp_image_operate(blocres_model, minus=blocres_half)
atomic_model = scipion.xmipp_pdb_label_from_volume(
    pdb=str(pdb_model),
    volume=fscq_diff,
    mask=mask,
    sampling=metadata.sampling,
    origin="%f %f %f" % (metadata.org_x, metadata.org_y, metadata.org_z),
)

[XMIPP] scipion run xmipp_image_operate -i /tmp/tmp_yga7l_h.map -o /tmp/tmpq0zu9lol.vol --minus /tmp/tmp7t3wchsc.map
Input File: /tmp/tmp_yga7l_h.map
Output File: /tmp/tmpq0zu9lol.vol
Xmipp command detected
Scipion v3.7.1 - Eugenius
[XMIPP] scipion run xmipp_pdb_label_from_volume -o /tmp/tmpy6l541qw.atom.pdb --pdb data/downloaded/pdb7l70.ent --vol /tmp/tmpq0zu9lol.vol --mask /tmp/tmpksqi23ou.mrc --sampling 0.835 --origin 0.000000 0.000000 0.000000
PDB file:           data/downloaded/pdb7l70.ent
Output:       /tmp/tmpy6l541qw.atom.pdb
Sampling rate:        0.835
Origin:               0.000000 0.000000 0.000000 
Radius:               0.8
mean value: = 0
absolute mean value: = 0
Xmipp command detected
Scipion v3.7.1 - Eugenius


In [12]:
with open(atomic_model.path) as f:
    print(f.read())

ATOM      1 N    PRO A  41     144.326 110.997 149.443  0.00110.02           N  
ATOM      2 CA   PRO A  41     144.800 109.921 150.321  0.00110.02           C  
ATOM      3 C    PRO A  41     145.274 110.455 151.673  0.00110.02           C  
ATOM      4 O    PRO A  41     146.294 111.142 151.720  0.00110.02           O  
ATOM      5 CB   PRO A  41     145.968 109.297 149.541  0.00110.02           C  
ATOM      6 CG   PRO A  41     146.140 110.109 148.294  0.00110.02           C  
ATOM      7 CD   PRO A  41     144.913 110.926 148.097  0.00110.02           C  
ATOM      8 N    PRO A  42     144.554 110.149 152.754  0.00109.82           N  
ATOM      9 CA   PRO A  42     144.937 110.695 154.064  0.00109.82           C  
ATOM     10 C    PRO A  42     146.265 110.122 154.533  0.00109.82           C  
ATOM     11 O    PRO A  42     146.505 108.915 154.446  0.00109.82           O  
ATOM     12 CB   PRO A  42     143.782 110.271 154.978  0.00109.82           C  
ATOM     13 CG   PRO A  42  